In [19]:
import random
import numpy
import fasttext
from nltk.corpus import stopwords
import csv
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import re

In [20]:
path_csv_file = "deneme.csv"

df = pd.read_csv(path_csv_file)
df.head()

,ictihat,Mahkeme
0,"Kasten basit yaralama suçundan sanık ...'nin,...",EMPTY
1,Taraflar arasında görülen ve yukarıda açı...,Asliye Hukuk Mahkemesi
2,"(5237 S. K. m. 53) (5271 S. K. m. 288, 289, 2...",EMPTY
3,BÖLGE ADLİYE MAHKEMESİ:... 5. Hukuk Dairesi ...,EMPTY
4,Taraflar arasında görülen davada; Davacı...,OTHERS


In [21]:
label_map = {}
count = 0
for label in df.Mahkeme.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.Mahkeme.map(label_map)
#print(label_map)
df.head()

,ictihat,Mahkeme,NUM_LABEL
0,"Kasten basit yaralama suçundan sanık ...'nin,...",EMPTY,0
1,Taraflar arasında görülen ve yukarıda açı...,Asliye Hukuk Mahkemesi,1
2,"(5237 S. K. m. 53) (5271 S. K. m. 288, 289, 2...",EMPTY,0
3,BÖLGE ADLİYE MAHKEMESİ:... 5. Hukuk Dairesi ...,EMPTY,0
4,Taraflar arasında görülen davada; Davacı...,OTHERS,2


In [22]:
df.isnull().sum()

ictihat      8
Mahkeme      0
NUM_LABEL    0
dtype: int64

In [23]:
df = df.dropna(how='any',axis=0)

In [24]:
df.isnull().sum()

ictihat      0
Mahkeme      0
NUM_LABEL    0
dtype: int64

In [25]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))    
    return [word for word in lower_case_only if word not in stopwords_tr]

In [26]:
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: " ".join(clean_text(x)))
df = df[['ictihat', 'NUM_LABEL']]
df

,ictihat,NUM_LABEL
0,kasten basit yaralama suçundan sanık sayılı tü...,0
1,taraflar arasında görülen yukarıda açıklanan d...,1
2,dosya gereği̇ görüşülüp dosya kapsamına temyiz...,0
3,bölge adli̇ye hukuk dairesi taraflar arasında ...,0
4,taraflar arasında görülen mirasbırakanları par...,2
...,...,...
27846,taraflar arasında görülen çekişme konusu ada p...,2
27847,taraflar arasında görülen mirasbırakan abisi p...,2
27848,taraflar arasında görülen kimliği belirsiz kiş...,2
27849,taraflar arasındaki davadan dolayı bölge adliy...,2


In [27]:
df_labeled = df.copy()
df_labeled['NUM_LABEL'] = '__label__' + df_labeled['NUM_LABEL'].astype(str)
df_labeled.head()

,ictihat,NUM_LABEL
0,kasten basit yaralama suçundan sanık sayılı tü...,__label__0
1,taraflar arasında görülen yukarıda açıklanan d...,__label__1
2,dosya gereği̇ görüşülüp dosya kapsamına temyiz...,__label__0
3,bölge adli̇ye hukuk dairesi taraflar arasında ...,__label__0
4,taraflar arasında görülen mirasbırakanları par...,__label__2


In [28]:
ictihat_label = df_labeled['NUM_LABEL'] + " " + df_labeled['ictihat']
ictihat_label

0        __label__0 kasten basit yaralama suçundan sanı...
1        __label__1 taraflar arasında görülen yukarıda ...
2        __label__0 dosya gereği̇ görüşülüp dosya kapsa...
3        __label__0 bölge adli̇ye hukuk dairesi tarafla...
4        __label__2 taraflar arasında görülen mirasbıra...
                               ...                        
27846    __label__2 taraflar arasında görülen çekişme k...
27847    __label__2 taraflar arasında görülen mirasbıra...
27848    __label__2 taraflar arasında görülen kimliği b...
27849    __label__2 taraflar arasındaki davadan dolayı ...
27850    __label__2 taraflar arasındaki davadan dolayı ...
Length: 27843, dtype: object

In [29]:
train, test = train_test_split(ictihat_label, test_size=0.2, random_state=42)

In [30]:
#write test and train into files
f_train = open("train.txt", "w", encoding='utf-8')
for i in range(len(numpy.array(train))):
    f_train.write(numpy.array(train)[i] + "\n")
f_train.close()

f_test = open("test.txt", "w",  encoding='utf-8')
for i in range(len(numpy.array(test))):
    f_test.write(numpy.array(test)[i] + "\n")
f_test.close()

In [31]:
model = fasttext.train_supervised(input="train.txt", epoch=10,  lr=0.5, loss='ova')

In [32]:
result = model.test("test.txt") #(n, precision, recall)
result

(5569, 0.8838211528101994, 0.8838211528101994)

In [33]:
test_file = "test.txt"

ValueError: Found input variables with inconsistent numbers of samples: [8, 3]

In [ ]:
#model.save_model("model_fasttext.bin")

In [ ]:
#model.predict("any_text")